# Filtering

Event filtering refers to the process of removing or extracting a subset of events based on some criterion such as the temperature of the measured sample at the time an event was detected.
Scipp's binned data can be used for this purpose.

Below, we describe two cases.
In the simple case the data contains the required coordinate and `sc.bin` can be used directly.
In the more complex case metadata requires preprocessing, and generally there are three steps to take:

1. Preprocess the metadata used for filtering.
   For example, a noisy time series of temperature values needs to converted into a series of time intervals with a fixed temperature value within the interval.
   This process might involve defining thresholds and tolerances or interpolation methods between measured temperature values.
2. Map event timestamps to temperature values.
3. Filter data based on temperature values.

## Preparation

We create some fake data for illustration purposes.

<div class="alert alert-info">

**Note**

In practice data to be filtered would be based on a loaded file. Details of this subsection can safely by skipped, as long as all cells are executed.

</div>

In [1]:
import numpy as np
import scipp as sc

<IPython.core.display.Javascript object>

In [2]:
np.random.seed(1) # Fixed for reproducibility
end_time = 100000
tof_max = 10000
width = tof_max/20
sizes = 4*np.array([7000, 3333, 3000, 5000])
size = np.sum(sizes)
data = sc.Variable(dims=['event'],
                   values=np.ones(size),
                   variances=np.ones(size))
time = sc.Variable(dims=['event'], unit=sc.units.s, dtype=sc.dtype.int64, shape=[size])
# time-of-flight in a neutron-scattering experiment
tof = sc.Variable(dims=['event'], unit=sc.units.us, dtype=sc.dtype.float64, shape=[size])
table = sc.DataArray(data=data, coords={'time':time, 'tof':tof})
table

<scipp.DataArray>
Dimensions: {{event, 73332}}
Coordinates:
  time                      int64      [s]              (event)  [0, 0, ..., 0, 0]
  tof                       float64    [µs]            (event)  [0.000000, 0.000000, ..., 0.000000, 0.000000]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

In [3]:
ntemp = 100
sample_temperature = sc.DataArray(
    data=sc.Variable(dims=['time'], unit=sc.units.K,
                     values=5*np.random.rand(100)+np.linspace(100, 120, num=ntemp)),
    coords={'time':sc.Variable(dims=['time'], unit=sc.units.s,
                               values=np.linspace(0, end_time, num=ntemp))})
x = sc.Variable(dims=['x'], unit=sc.units.m, values=np.linspace(0, 1, num=4))

end = sc.Variable(dims=['x'], values=np.cumsum(sizes))
begin = end.copy()
begin.values -= sizes
events = sc.DataArray(
    data=sc.bins(begin=begin, end=end, dim='event', data=table),
    coords={'x': x},
    attrs={'sample_temperature': sc.Variable(value=sample_temperature)})
for size, bucket in zip(sizes, events.values):
    bucket.coords['time'].values = np.linspace(0, end_time, num=size)
    bucket.coords['tof'].values = np.concatenate(
        (np.concatenate(
            (7*width + width*np.random.randn(size//4),
             13*width + width*np.random.randn(size//4))),
         10*width + width*np.random.randn(size//2)))
events

<scipp.DataArray>
Dimensions: {{x, 4}}
Coordinates:
  x                         float64    [m]              (x)  [0.000000, 0.333333, 0.666667, 1.000000]
Data:
                            DataArrayView  [dimensionless]  (x)  [<scipp.DataArrayView>
Dimensions: {{event, 28000}}
Coordinates:
  time                      int64      [s]              (event)  [0, 3, ..., 99996, 100000]
  tof                       float64    [µs]            (event)  [3657.817474, 2488.899392, ..., 4940.780540, 5328.824996]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 13332}}
Coordinates:
  time                      int64      [s]              (event)  [0, 7, ..., 99992, 100000]
  tof                       float64    [µs]            (event)  [4095.085717, 3596.569337, ..., 5866.263268, 5605.491936]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 12000}}
Coordinates:
  time                      int64      [s]              (event)  [0, 8, ..., 99991, 100000]
  tof                       float64    [µs]            (event)  [4099.616153, 2460.326481, ..., 4803.603079, 5116.263634]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 20000}}
Coordinates:
  time                      int64      [s]              (event)  [0, 5, ..., 99994, 100000]
  tof                       float64    [µs]            (event)  [3640.648981, 3293.735734, ..., 5004.568230, 5290.823433]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

]
Attributes:
  sample_temperature        DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 100}}
Coordinates:
  time                      float64    [s]              (time)  [0.000000, 1010.101010, ..., 98989.898990, 100000.000000]
Data:
                            float64    [K]              (time)  [102.085110, 103.803643, ..., 119.812331, 123.085725]

]

## Filtering based on existing coords

### Extracting data based on an interval

We can use `sc.bin` with the desired bounds to extract all data points (events) that have coord values falling within an interval:

In [4]:
tof_interval = sc.Variable(dims=['tof'], values=[2000.0, 3000.0], unit=sc.units.us)
filtered = sc.bin(events, edges=[tof_interval])
filtered

<scipp.DataArray>
Dimensions: {{x, 4}, {tof, 1}}
Coordinates:
  tof                       float64    [µs]            (tof [bin-edge])  [2000.000000, 3000.000000]
  x                         float64    [m]              (x)  [0.000000, 0.333333, 0.666667, 1.000000]
Data:
                            DataArrayView  [dimensionless]  (x, tof)  [<scipp.DataArrayView>
Dimensions: {{event, 1094}}
Coordinates:
  time                      int64      [s]              (event)  [3, 107, ..., 24983, 93331]
  tof                       float64    [µs]            (event)  [2488.899392, 2952.544077, ..., 2926.639219, 2883.417601]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 564}}
Coordinates:
  time                      int64      [s]              (event)  [30, 90, ..., 24986, 97734]
  tof                       float64    [µs]            (event)  [2597.438224, 2882.813133, ..., 2926.807234, 2955.755708]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 440}}
Coordinates:
  time                      int64      [s]              (event)  [8, 16, ..., 24852, 61221]
  tof                       float64    [µs]            (event)  [2460.326481, 2193.954433, ..., 2232.848621, 2979.159780]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 787}}
Coordinates:
  time                      int64      [s]              (event)  [30, 45, ..., 24986, 94904]
  tof                       float64    [µs]            (event)  [2713.515033, 2782.407451, ..., 2962.543976, 2836.556360]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

]
Attributes:
  sample_temperature        DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 100}}
Coordinates:
  time                      float64    [s]              (time)  [0.000000, 1010.101010, ..., 98989.898990, 100000.000000]
Data:
                            float64    [K]              (time)  [102.085110, 103.803643, ..., 119.812331, 123.085725]

]

### Extracting/splitting data based on multiple intervals

In the same manner, we can extract data with a list of (adjacent) intervals:

In [5]:
tof_intervals = sc.Variable(dims=['tof'], values=np.linspace(2000, 3000, num=4), unit=sc.units.us)
filtered = sc.bin(events, edges=[tof_intervals])
filtered

<scipp.DataArray>
Dimensions: {{x, 4}, {tof, 3}}
Coordinates:
  tof                       float64    [µs]            (tof [bin-edge])  [2000.000000, 2333.333333, 2666.666667, 3000.000000]
  x                         float64    [m]              (x)  [0.000000, 0.333333, 0.666667, 1.000000]
Data:
                            DataArrayView  [dimensionless]  (x, tof)  [<scipp.DataArrayView>
Dimensions: {{event, 56}}
Coordinates:
  time                      int64      [s]              (event)  [275, 635, ..., 24443, 24875]
  tof                       float64    [µs]            (event)  [2282.581118, 2103.457500, ..., 2252.316044, 2179.651371]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 265}}
Coordinates:
  time                      int64      [s]              (event)  [3, 178, ..., 24850, 24975]
  tof                       float64    [µs]            (event)  [2488.899392, 2571.009068, ..., 2415.412519, 2615.169329]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, ..., <scipp.DataArrayView>
Dimensions: {{event, 182}}
Coordinates:
  time                      int64      [s]              (event)  [55, 435, ..., 24546, 24766]
  tof                       float64    [µs]            (event)  [2665.113452, 2638.096833, ..., 2492.916474, 2554.216191]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 558}}
Coordinates:
  time                      int64      [s]              (event)  [30, 45, ..., 24986, 94904]
  tof                       float64    [µs]            (event)  [2713.515033, 2782.407451, ..., 2962.543976, 2836.556360]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

]
Attributes:
  sample_temperature        DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 100}}
Coordinates:
  time                      float64    [s]              (time)  [0.000000, 1010.101010, ..., 98989.898990, 100000.000000]
Data:
                            float64    [K]              (time)  [102.085110, 103.803643, ..., 119.812331, 123.085725]

]

Events in each of the subintervals can then be accessed using the usual slicing syntax:

In [6]:
filtered['tof',2]

<scipp.DataArrayView>
Dimensions: {{x, 4}}
Coordinates:
  x                         float64    [m]              (x)  [0.000000, 0.333333, 0.666667, 1.000000]
Data:
                            DataArrayView  [dimensionless]  (x)  [<scipp.DataArrayView>
Dimensions: {{event, 773}}
Coordinates:
  time                      int64      [s]              (event)  [107, 128, ..., 24983, 93331]
  tof                       float64    [µs]            (event)  [2952.544077, 2813.441340, ..., 2926.639219, 2883.417601]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 404}}
Coordinates:
  time                      int64      [s]              (event)  [90, 150, ..., 24986, 97734]
  tof                       float64    [µs]            (event)  [2882.813133, 2701.765061, ..., 2926.807234, 2955.755708]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 308}}
Coordinates:
  time                      int64      [s]              (event)  [50, 100, ..., 24718, 61221]
  tof                       float64    [µs]            (event)  [2770.815179, 2771.941197, ..., 2784.801523, 2979.159780]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

, <scipp.DataArrayView>
Dimensions: {{event, 558}}
Coordinates:
  time                      int64      [s]              (event)  [30, 45, ..., 24986, 94904]
  tof                       float64    [µs]            (event)  [2713.515033, 2782.407451, ..., 2962.543976, 2836.556360]
Data:
                            float64    [dimensionless]  (event)  [1.000000, 1.000000, ..., 1.000000, 1.000000]  [1.000000, 1.000000, ..., 1.000000, 1.000000]

]
Attributes:
  sample_temperature        DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{time, 100}}
Coordinates:
  time                      float64    [s]              (time)  [0.000000, 1010.101010, ..., 98989.898990, 100000.000000]
Data:
                            float64    [K]              (time)  [102.085110, 103.803643, ..., 119.812331, 123.085725]

]
  tof                       float64    [µs]            (tof [bin-edge])  [2666.666667, 3000.000000]

## Filtering based on arbitrary metadata
### Step 1: Preprocess metadata

Our data contains a coordinate with metadata related to the temperature of the measured sample:

In [7]:
timeseries = events.attrs['sample_temperature'].value
sc.plot(timeseries)

This is a timeseries with noisy measurements, as could be obtained, e.g., from a temperature sensor.
For event filtering we require intervals with a fixed temperature.
This can be obtained in many ways.
In this example we do so by taking the mean over subintervals:

In [9]:
average=4
temperature = sc.split(timeseries, 'time', {'time': ntemp//average, 'dummy': average})
temperature.coords['time'] = sc.plotting.tools.to_bin_edges(temperature.coords['time']['dummy', 0], 'time')
temperature = sc.mean(temperature, 'dummy')
sc.plot(temperature)

<div class="alert alert-info">

**Note**

We are using integer data with a unit of seconds for the time series since scipp has no support for datetime64 yet.

</div>

### Step 2: Map time stamps

The `temperature` data array computed above can be seen as a discretized functional dependence of temperature on time.
This "function" can now be used to map the `time` of each event to the `temperature` of each event:

In [ ]:
event_temp = sc.buckets.map(temperature, events.data, 'time')
events.bins.data.coords['temperature'] = event_temp.bins.data

The event lists with temperature values created by `scipp.map` have been added as a new coordinate:

In [ ]:
events.values[0]

### Step 3: Filter with `scipp.bin`

The temperature coordinate created in the previous step can now be used for the actual filtering step.
With a `temperature` coordinate stored as part of `events` it is possible to use `scipp.bin` with temperature bins:

In [ ]:
tof_bins = sc.Variable(dims=['tof'], unit=sc.units.us, values=np.linspace(0,tof_max,num=100))
temp_bins = sc.Variable(dims=['temperature'], unit=sc.units.K, values=np.linspace(100.0, 130.0, num=6))
binned_events = sc.bin(events, [temp_bins, tof_bins])
binned_events

Filtering is then performed by slicing and, if desired, copying:

In [ ]:
filtered_view = binned_events['temperature', 0:3] # view containing only relevant events
filtered = binned_events['temperature', 0:3].copy() # extract only relevant events by copying

Slicing combined with histogramming also performs a filter operation since all events outside the histogram bounds are dropped:

In [ ]:
sc.plot(binned_events['temperature', 1])

In [ ]:
sc.plot(binned_events['temperature', 3])

Results from filter operations can also be inserted into a dataset for convenient handling of further operations such as histogramming, summing, or plotting:

In [ ]:
d = sc.Dataset()
d['below_T_c'] = binned_events['temperature', 1]
d['above_T_c'] = binned_events['temperature', 3]
sc.plot(sc.sum(d.bins.sum(), 'x'))

We can also bin without the time-of-flight coordinate to obtain the temperature dependence of the total event count, e.g., for normalization purposes:

In [ ]:
binned_events = sc.bin(events, [temp_bins])
sc.plot(binned_events)